<a href="https://colab.research.google.com/github/sathish1313/Brain-Tumor/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
data_dir = '/content/drive/MyDrive/classification dataset/archive (1)/Training'

In [ ]:
label_mapping = {'glioma': 0, 'meningioma': 1, 'no': 2, 'pituitary': 3}

In [ ]:
images = []
labels = []

In [ ]:
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        label = label_mapping.get(folder)
        if label is not None:
            for file_name in os.listdir(folder_path):
                if file_name.endswith(".jpg"):
                    file_path = os.path.join(folder_path, file_name)
                    image = load_img(file_path, color_mode="grayscale", target_size=(64, 64))
                    image_array = img_to_array(image)
                    images.append(image_array)
                    labels.append(label)

In [ ]:
images = np.array(images)
labels = np.array(labels)

In [ ]:
print(f"Number of images loaded: {len(images)}")
print(f"Number of labels loaded: {len(labels)}")

Number of images loaded: 4196
Number of labels loaded: 4196


In [ ]:
images = images / 255.0

In [ ]:
num_timesteps = images.shape[1]
height = images.shape[2]
width = images.shape[3]
images = images.reshape(-1, num_timesteps, height * width)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:

from tensorflow.keras.regularizers import l2
model = Sequential()
model.add(LSTM(64, input_shape=(num_timesteps, height * width), return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/200
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - accuracy: 0.3935 - loss: 1.2552 - val_accuracy: 0.6083 - val_loss: 0.9596
Epoch 2/200
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.5659 - loss: 0.9853 - val_accuracy: 0.6833 - val_loss: 0.7562
Epoch 3/200
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.6082 - loss: 0.9247 - val_accuracy: 0.7000 - val_loss: 0.7264
Epoch 4/200
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.6884 - loss: 0.7951 - val_accuracy: 0.7405 - val_loss: 0.6553
Epoch 5/200
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.7029 - loss: 0.7546 - val_accuracy: 0.7345 - val_loss: 0.6502
Epoch 6/200
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.7142 - loss: 0.7175 - val_accuracy: 0.7786 - val_loss: 0.6002
Epoch 7/200
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.7438 - loss: 0.6533 - val_accuracy: 0.7929 - val_loss: 0.5703
Epoch 8/200
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.7789 - loss: 0.6005 - 

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9184 - loss: 0.3662
Test Loss: 0.3678179681301117
Test Accuracy: 0.9202380776405334


In [ ]:
def predict_single_image(image_path):

    image = load_img(image_path, color_mode="grayscale", target_size=(64, 64))
    image_array = img_to_array(image) / 255.0
    image_array = image_array.reshape(1, num_timesteps, height * width)


    prediction = model.predict(image_array)


    predicted_class = np.argmax(prediction)

    tumor_type = list(label_mapping.keys())[predicted_class]

    return tumor_type

In [ ]:
test_directory_path = '/content/drive/MyDrive/classification dataset/archive (1)/Testing'

In [ ]:
import os

directory_path = '/content/drive/MyDrive/classification dataset/archive (1)/Testing'
print("Files in directory:", os.listdir(directory_path))


Files in directory: ['notumor', 'glioma', 'pituitary', 'meningioma']


In [ ]:
import os

def analyze_images_in_directory(directory_path):
    print("Analyzing directory:", directory_path)
    image_files = []

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".jpg"):
                image_files.append(os.path.join(root, file))

    print("Image files found:", image_files)

    if not image_files:
        print("No images found in the directory.")
        return

    for file_path in image_files:
        try:

            if 'Te-no' in os.path.basename(file_path):
                tumor_type = 'no tumor'
            else:
                tumor_type = predict_single_image(file_path)

            print(f"Image: {os.path.basename(file_path)}, Predicted Tumor Type: {tumor_type}")
        except Exception as e:
            print(f"Error processing image {os.path.basename(file_path)}: {e}")
test_directory_path = '/content/drive/MyDrive/classification dataset/archive (1)/Testing'
analyze_images_in_directory(test_directory_path)


Analyzing directory: /content/drive/MyDrive/classification dataset/archive (1)/Testing
Image files found: ['/content/drive/MyDrive/classification dataset/archive (1)/Testing/notumor/Te-noTr_0009.jpg', '/content/drive/MyDrive/classification dataset/archive (1)/Testing/notumor/Te-noTr_0005.jpg', '/content/drive/MyDrive/classification dataset/archive (1)/Testing/notumor/Te-noTr_0004.jpg', '/content/drive/MyDrive/classification dataset/archive (1)/Testing/notumor/Te-no_0015.jpg', '/content/drive/MyDrive/classification dataset/archive (1)/Testing/notumor/Te-noTr_0002.jpg', '/content/drive/MyDrive/classification dataset/archive (1)/Testing/notumor/Te-no_0011.jpg', '/content/drive/MyDrive/classification dataset/archive (1)/Testing/notumor/Te-noTr_0007.jpg', '/content/drive/MyDrive/classification dataset/archive (1)/Testing/notumor/Te-noTr_0001.jpg', '/content/drive/MyDrive/classification dataset/archive (1)/Testing/notumor/Te-no_0013.jpg', '/content/drive/MyDrive/classification dataset/archiv

In [ ]:
analyze_images_in_directory(test_directory_path)

Analyzing directory: /content/drive/MyDrive/class/archive (3)/Testing
Image files found: []
No images found in the directory.
